# Json to CSV for survey data

In [1]:
import json
import math
import csv
import numpy as np

In [29]:

# Turn the json file into a dictionary
jsonFile = open('../data/surveyresults_5-7-21.json')
d = json.load(jsonFile)

# Loop through the dictionary and make a csv
headerRow = [
    'id', 
    'preferred app',
    'app use frequency',
    'experience on app',
    'app inaccuracy',
    'app reliable',
    'stats experience',
    'vis type', #TODO fix order of trial headers
    'question type',
    'guess', 
    'true precipitation amount',
    'true pop',
    'confidence', # confidence in their answer
    'follow up text',
    'effort bar',
    'effort dot',
    'effort hop',
    'effort text',
    'rank1',
    'rank2',
    'rank3',
    'rank4',
    'feedback',
]
# headerRow = ['id','Education','Field','Statistics Experience','Data vis experience','trial vis type','trial reported percent','trial true percent','trial log error']

d = d['data']
with open('output.csv', 'w', newline='') as csvfile:
    csvWriter = csv.writer(csvfile, delimiter=',')
    csvWriter.writerow(headerRow)
    i =1
    for key in d:
        # print('person', i, 'key', key)
        i = i+1
        row = []
        # get id
        row.append(key)

        # get demographic info
        demographic = d[key]['demographic']
        row.append(demographic['app'])
        row.append(demographic['frequency'])
        row.append(demographic['experience'])
        row.append(demographic['inaccurate'])
        if ('reliable' in demographic.keys()):
            row.append(demographic['reliable'])
        else:
            row.append(demographic['Reliable'])
        row.append(demographic['stats'])

        # get exit survey results
        exitSurveyResults = d[key]['exitSurveyResults']
        row.append(exitSurveyResults['EffortBar'])
        row.append(exitSurveyResults['EffortDot'])
        row.append(exitSurveyResults['EffortHOP'])
        row.append(exitSurveyResults['EffortText'])
        row.append(exitSurveyResults['Rank1'])
        row.append(exitSurveyResults['Rank2'])
        row.append(exitSurveyResults['Rank3'])
        row.append(exitSurveyResults['Rank4'])
        row.append(exitSurveyResults['Feedback'])

        # get trial data
        for trial in d[key]['trials']:
            if ('done' not in trial.keys()):
                # print(type(trial))
                subRow = []
                # id
                subRow.append(row[0])

                # demographics
                subRow.append(row[1])
                subRow.append(row[2])
                subRow.append(row[3])
                subRow.append(row[4])
                subRow.append(row[5])
                subRow.append(row[6])
                # trial information
                # TODO finish this section
                subRow.append(trial['type'])
                subRow.append(trial['questionType'])
                # print(trial['mainAnswer'])
                subRow.append(trial['mainAnswer'])
                # TODO add correct answer

                true_values_pop, true_values_amount = true_values(trial['points'][0], trial['points'][1], trial['questionType'])
                subRow.append(true_values_pop)
                subRow.append(true_values_amount)
                subRow.append(trial['confidence'])
                subRow.append(trial['followUpText'])

                # truePercent = trial['low']/trial['high']
                # subRow.append(truePercent)
                # logError = math.log((abs((truePercent*100)-(float(trial['guess'])*100))+0.125),2)
                # subRow.append(logError)
                # print(subRow)

                # "askedForHelp" : false,
                # "confidence" : "Very unsure",
                # "followUpText" : "",
                # "mainAnswer" : "0.3",
                # "points" : [ [ 0.45, 0.25, 0.3, 0.35, 0.3, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0, 0, 0.05, 0.2, 0.3, 0.2, 0.1, 0.45, 0.3, 0.1, 0.3, 0.35, 0.5 ], [ 0.7, 0.65, 0.7, 0.55, 0.55, 0.35, 0.3, 0.5, 0.3, 0.2, 0.15, 0.05, 0, 0.1, 0.3, 0.4, 0.45, 0.55, 0.75, 0.8, 0.7, 0.8, 0.9, 0.95 ] ],
                # "questionType" : "amount",
                # "type" : "Hypothetical Outcome Plot"
                
                # exit survey
                # subRow.append(row[6])
                subRow.append(row[7])
                subRow.append(row[8])
                subRow.append(row[9])
                subRow.append(row[10])
                subRow.append(row[11])
                subRow.append(row[12])
                subRow.append(row[13])
                subRow.append(row[14])
                subRow.append(row[15])
                # print(subRow)
                csvWriter.writerow(subRow)

In [21]:

'''
Return the true prediction and true chance of precipitation for a
specific trial question. The true values can either be a point
value or a weighted average, based on the type of question.
'''
def true_values(all_predictions, all_chances, question_type):
    if (question_type == 'friend' or question_type == 'alone' or question_type == 'event'):
        return data_avg(all_predictions, all_chances, question_type)
        

    elif (question_type == 'amount' or question_type == 'probability'):
        return [all_predictions[15], all_chances[15]]

    else: print('Error: Not a trial')
        

'''
Calculate the weighted average prediction and weighted average chance
of precipitation based on the type of viz
'''
def data_avg(all_amount_predictions, all_chances, question_type):

    n_hours = 24
    pop_weighted_avg = 0
    amt_weighted_avg = 0
    # before_sum = 0
    # after_sum = 0
    # during_value = 0
    index_3pm = 15

    # check weather 2 hrs before to see if ground is wet
    # and 4 hrs after bc hiking will probs take that long
    if question_type == 'friend' or question_type == 'alone':
        amt_weighted_avg = np.mean(all_amount_predictions[index_3pm-2:index_3pm+2])
        pop_weighted_avg = np.mean(all_chances[index_3pm-2:index_3pm+2])

    elif question_type == 'event':
        amt_weighted_avg = np.mean(all_amount_predictions)
        pop_weighted_avg = np.mean(all_chances)

    return pop_weighted_avg, amt_weighted_avg

# Data Analysis

In [2]:
import numpy as np
import pandas as pd


In [10]:
data = pd.read_csv('output.csv')

## Plot things per vis type

plot effort by vis type

In [ ]:
import plotly.express as px
# data_canada = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(data_canada, x='year', y='pop')
fig.show()


In [5]:
import plotly.express as px
# df = px.data.tips()
fig = px.histogram(data, x="effort bar")
fig.show()


ModuleNotFoundError: No module named 'plotly'

In [50]:
vis_types = set(data['vis type'])
confidence_categories = set(data['confidence'])
confidence = []
for vis_type in vis_types:
    subset = data.loc[data['vis type'] == vis_type]
    len(text_data.loc[text_data['confidence'] == 'Kind of confident'])

{'Hypothetical Outcome Plot', 'Quantile Dot Plot', 'Static Bar Chart', 'Text'}


15

In [27]:
cleanup = {"confidence_numeric":     {"Very Confident": 5,
                                        "Kind of confident": 4, 
                                        "Neutral": 3,
                                        "Unsure": 2,
                                        "So unsure": 1,
                                        "Very unsure": 0},
                                        }

data = data.replace(cleanup)

In [33]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks", color_codes=True)

len(data.loc[''])

plt.hist(data=data, X='vis type', y='confidence_numeric')

KeyError: ''